In [10]:
%matplotlib inline
from detail import Detail
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
import pylab
import json
import random
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [11]:
annFile='../../json/PASCAL_trainval_merged.json' # annotations
imgDir='../../VOCdevkit/VOC2010/JPEGImages' # jpeg images
phase='trainval'

In [12]:
# initialize detail api for instance annotations
detailsGt = Detail(annFile, imgDir, phase)

print('\n***info***')
detailsGt.info()

loading annotations into memory...
Done (t=7.63s)
creating index...
index created! 7.75s

***info***
description: This is the 3.0 version of the CVPR 2017 PASCAL in Detail dataset.
version: 3.0
year: 2017
contributor: https://sites.google.com/view/pasd
date_created: 2017-07-18


## Load eval module for keypoints

In [13]:
from detail import detaileval_kpt
from importlib import reload
reload(detaileval_kpt)
detaileval = detaileval_kpt.DetailEvalKpt(detailsGt)

Total category #1, total img #3547 


## Fake a result json for evaluation

In [19]:
gt_imgs_human = detailsGt.getImgs(cats=['person']);
detailRes = []
count = 0
for gt_img in gt_imgs_human:
    '''
    if(2 in gt_img['categories']):
        detailRes[-1]['category_score'][0] = 1
    if(23 in gt_img['categories'] and count < 200):
        count += 1
        detailRes[-1]['category_score'][1] = 1
    '''
    for k in gt_img['keypoints']:
        detailRes.append(
            {'image_id': gt_img['image_id'], 
             'score': 1, 
             'category_id': 284,
             'keypoints': np.ndarray.tolist(np.zeros(42))})
        detailRes[-1]['keypoints'] = k['keypoints']
        #cat_idx = [k]
        #detailRes[-1]['category_score'][cat_idx] = 1
    
result = {}
result['pred'] = detailRes
detail_res_file = '../res/detail_kpt_fake_result.json'
json.dump(result, open(detail_res_file, 'w'), indent = 4)

In [20]:
# start evaluation
from detail import detaileval_kpt
from importlib import reload
reload(detaileval_kpt)
detaileval = detaileval_kpt.DetailEvalKpt(detailsGt)
detailRes = json.load(open('../res/detail_kpt_fake_result.json', 'r'))['pred']
detaileval.loadRes(detailRes)

Total category #1, total img #3547 


In [16]:
detaileval.evaluate()

DONE (t=2.27s).


In [17]:
detaileval.accumulate()

Accumulating evaluation results...
1 1 3
DONE (t=0.12s).


In [18]:
detaileval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.990
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.990
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.962
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.989
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 1.000
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 1.000
